In [ ]:
import pandas as pd
import os
import random
from datetime import datetime, timedelta
from env import mod_path, ntfy_nofi

In [ ]:
df_rans = pd.DataFrame()
df_rotk = pd.DataFrame()
df_botk = pd.DataFrame()

In [ ]:
column_names = [
    "time",
    "source",
    "attackName",
    "attackKind",
    "level",
    "agentName",
    "agentId",
    "processGuid",
    "processId",
    "image",
    "user",
    # nullable
    "triage",
    # 기타 필드들 통합 내용. 공통 필드이외의 내용을 이 필드에 통합해서 리턴함. 필드를 구분해서 보려면 상세 조회 필요
    "content",
]


In [ ]:
data_rans = [
    [
        "2023-07-01T00:50:45.187390204+00:00",
        "collect",
        "Alcatraz",
        "Ransomware",
        4,
        "DESKTOP-0FS3LG3",
        "f98ce9dc-4df6-445a-890d-c7a5368c230d",
        "{145e89fa-6e99-64b7-261c-000000000800}",
        "19924",
        "C:\\Users\\vboxuser\\Downloads\\53292899fec.exe",
        "DESKTOP-0FS3LG3\spdlq",
        "null",
        "-",
    ]
]

In [ ]:
data_rotk = [
    [
        "2023-07-01T00:50:45.187390204+00:00",
        "collect",
        "R77",
        "Rootkit",
        2,
        "DESKTOP-0FS3LG3",
        "f98ce9dc-4df6-445a-890d-c7a5368c230d",
        "{145e89fa-6e99-64b7-261c-000000000800}",
        "34721",
        "C:\\Users\\vboxuser\\Downloads\\ilovepdf_pages-to-jpg\\pdf_sample.exe",
        "DESKTOP-0FS3LG3\spdlq",
        "null",
        "-",
    ]
]

In [ ]:
data_botk = [
    [
        "2023-07-01T00:50:45.187390204+00:00",
        "collect",
        "ESPecter",
        "Bootkit",
        3,
        "DESKTOP-0FS3LG3",
        "f98ce9dc-4df6-445a-890d-c7a5368c230d",
        "{145e89fa-6e99-64b7-261c-000000000800}",
        "27605",
        "C:\\Program Files\\NVIDIA Corporation\\NvContainer\\bootloader.exe",
        "DESKTOP-0FS3LG3\spdlq",
        "null",
        "-",
    ]
]

In [ ]:
df_rans = pd.DataFrame(data=data_rans, columns=column_names)
df_rotk = pd.DataFrame(data=data_rotk, columns=column_names)
df_botk = pd.DataFrame(data=data_botk, columns=column_names)

In [ ]:
df_rans_gen = pd.concat([df_rans] * 200, ignore_index=True)
df_rotk_gen = pd.concat([df_rotk] * 200, ignore_index=True)
df_botk_gen = pd.concat([df_botk] * 200, ignore_index=True)

In [ ]:
def random_date(start, end):
    return start + timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())))

In [ ]:
def random_datetime_with_nanoseconds(start, end):
    random_dt = random_date(start, end)
    # Generate a random nanoseconds value (0 to 999,999,999)
    nanoseconds = random.randint(0, 999_999_999)
    # Combine the datetime and the nanoseconds
    formatted_dt = f"{random_dt.isoformat()}.{str(nanoseconds).zfill(9)}+00:00"
    return formatted_dt

In [ ]:
start_date = datetime(2023, 7, 1)
end_date = datetime(2023, 7, 31)

In [ ]:
rans_arr = []
for i in range(0, 200):
    formatted_datetime_ns = random_datetime_with_nanoseconds(start_date, end_date)
    rans_arr.append(formatted_datetime_ns)
rans_arr.sort()

In [ ]:
rotk_arr = []
for i in range(0, 200):
    formatted_datetime_ns = random_datetime_with_nanoseconds(start_date, end_date)
    rotk_arr.append(formatted_datetime_ns)
rotk_arr.sort()

In [ ]:
botk_arr = []
for i in range(0, 200):
    formatted_datetime_ns = random_datetime_with_nanoseconds(start_date, end_date)
    botk_arr.append(formatted_datetime_ns)
botk_arr.sort()

In [ ]:
time_df_rans = pd.DataFrame(rans_arr, columns=['time'])
time_df_rotk = pd.DataFrame(rotk_arr, columns=['time'])
time_df_botk = pd.DataFrame(botk_arr, columns=['time'])

In [ ]:
time_df_rans_temp = pd.DataFrame()
time_df_rans_temp['date'] = pd.to_datetime(time_df_rans['time']).dt.date
time_df_rans_count = time_df_rans_temp.groupby('date').size().reset_index(name='rans count')

In [ ]:
time_df_rotk_temp = pd.DataFrame()
time_df_rotk_temp['date'] = pd.to_datetime(time_df_rotk['time']).dt.date
time_df_rotk_count = time_df_rotk_temp.groupby('date').size().reset_index(name='rotk count')

In [ ]:
time_df_botk_temp = pd.DataFrame()
time_df_botk_temp['date'] = pd.to_datetime(time_df_botk['time']).dt.date
time_df_botk_count = time_df_botk_temp.groupby('date').size().reset_index(name='botk count')

In [ ]:
merged_count = pd.merge(time_df_rans_count, time_df_rotk_count, on='date').merge(time_df_botk_count, on='date')
merged_count['per day count'] = merged_count['rans count'] + merged_count['rotk count'] + merged_count['botk count']
merged_count

In [ ]:
df_rans_gen.time = time_df_rans
df_rotk_gen.time = time_df_rotk
df_botk_gen.time = time_df_botk

In [ ]:
# output of csv
df_rans_gen.to_csv(mod_path + "detect/ransonware_alcatraz.csv", index=False, sep="\t")
df_rotk_gen.to_csv(mod_path + "detect/rootkit_r77.csv", index=False, sep="\t")
df_botk_gen.to_csv(mod_path + "detect/bootkit_especter.csv", index=False, sep="\t")

In [ ]:
# output of json
df_rans_gen.to_json(mod_path + "detect/ransonware_alcatraz.json", orient='records', date_format='iso')
df_rotk_gen.to_json(mod_path + "detect/rootkit_r77.json", orient='records', date_format='iso')
df_botk_gen.to_json(mod_path + "detect/bootkit_especter.json", orient='records', date_format='iso')
